# Orbit
## 1. Setup
### 1.1 Config

In [8]:
# MLflow model path
model_path = "model"

# prediction config for pyfunc flavor
pyfunc_predict_conf = {
        "decompose": True,
        "seed": 43,
}

### 1.2 Imports

In [32]:
import mlflow
import mlflow_flavors
from mlflow.models import infer_signature
from orbit.models import DLT
from orbit.utils.dataset import load_iclaims
import pandas as pd

### 1.3 Load sample data

In [6]:
df = load_iclaims()

# train-test split
test_size = 52
train_df = df[:-test_size]
test_df = df[-test_size:]

test_df.head()

,week,claims,trend.unemploy,trend.filling,trend.job,sp500,vix
391,2017-07-02,12.440694,0.219882,0.104405,-0.079668,0.332886,-0.359894
392,2017-07-09,12.557887,0.252143,0.032946,0.029532,0.346844,-0.522570
393,2017-07-16,12.459796,0.186546,0.069313,-0.005560,0.352226,-0.538469
394,2017-07-23,12.303449,0.169451,0.032946,0.029532,0.352048,-0.443742
395,2017-07-30,12.199934,0.186546,0.051295,0.052260,0.353959,-0.469334


## 2. Example usage of native `orbit` flavor and `pyfunc` flavor

### 2.1 Train and save model

In [12]:
with mlflow.start_run():

    dlt = DLT(
        response_col="claims",
        date_col="week",
        regressor_col=["trend.unemploy", "trend.filling", "trend.job"],
        seasonality=52,
    )
    dlt.fit(df=train_df)
    dlt.pyfunc_predict_conf = pyfunc_predict_conf

    mlflow_flavors.orbit.save_model(dlt, model_path)

INFO:orbit:Sampling (PyStan) with chains: 4, cores: 8, temperature: 1.000, warmups (per chain): 225 and samples(per chain): 25.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)
2023/01/06 13:39:23 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: model/model.pkl, flavor: orbit), fall back to return ['orbit==1.1.3']. Set logging level to DEBUG to see the full traceback.
/home/benjamin/PycharmProjects/mlflow_flavors/.venv/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


### 2.2 Load model
#### 2.2.1 Native orbit flavor

In [13]:
loaded_model = mlflow_flavors.orbit.load_model(model_path)

#### 2.2.1 Pyfunc flavor

In [14]:
loaded_pyfunc = mlflow_flavors.orbit.pyfunc.load_model(model_path)

### 2.3 Generate predictions
#### 2.3.1 Native orbit flavor

In [18]:
loaded_model.predict(test_df, decompose=True, seed=43).head()

,week,prediction_5,prediction,prediction_95,trend_5,trend,trend_95,seasonality_5,seasonality,seasonality_95,regression_5,regression,regression_95
0,2017-07-02,12.329969,12.415831,12.505553,12.249114,12.337002,12.435567,0.023342,0.054511,0.076124,0.004599,0.026254,0.039776
1,2017-07-09,12.426513,12.534631,12.623807,12.234586,12.345293,12.444151,0.146387,0.179109,0.204080,0.007272,0.014737,0.022364
2,2017-07-16,12.299631,12.376752,12.470228,12.246894,12.329269,12.441898,-0.000433,0.031216,0.051681,0.006270,0.016838,0.024393
3,2017-07-23,12.121754,12.222322,12.329275,12.222615,12.331567,12.453743,-0.143769,-0.112244,-0.092680,0.004225,0.010565,0.017122
4,2017-07-30,12.077165,12.177895,12.287542,12.218419,12.322874,12.428633,-0.184385,-0.154620,-0.132869,0.004059,0.011917,0.020557


#### 2.3.1 Pyfunc flavor

In [17]:
loaded_pyfunc.predict(test_df).head()

,week,prediction_5,prediction,prediction_95,trend_5,trend,trend_95,seasonality_5,seasonality,seasonality_95,regression_5,regression,regression_95
0,2017-07-02,12.329969,12.415831,12.505553,12.249114,12.337002,12.435567,0.023342,0.054511,0.076124,0.004599,0.026254,0.039776
1,2017-07-09,12.426513,12.534631,12.623807,12.234586,12.345293,12.444151,0.146387,0.179109,0.204080,0.007272,0.014737,0.022364
2,2017-07-16,12.299631,12.376752,12.470228,12.246894,12.329269,12.441898,-0.000433,0.031216,0.051681,0.006270,0.016838,0.024393
3,2017-07-23,12.121754,12.222322,12.329275,12.222615,12.331567,12.453743,-0.143769,-0.112244,-0.092680,0.004225,0.010565,0.017122
4,2017-07-30,12.077165,12.177895,12.287542,12.218419,12.322874,12.428633,-0.184385,-0.154620,-0.132869,0.004059,0.011917,0.020557


## 3. Model deployment example
### 3.1 Create experiment

In [28]:
mlflow.set_experiment("Test Orbit")

with mlflow.start_run() as run:

    dlt = DLT(
        response_col="claims",
        date_col="week",
        regressor_col=["trend.unemploy", "trend.filling", "trend.job"],
        seasonality=52,
    )
    dlt.fit(df=train_df)
    dlt.pyfunc_predict_conf = pyfunc_predict_conf
    
    # Need to include a model signature, so MLflow can automatically decode datetime from JSON
    prediction = dlt.predict(test_df)
    signature = infer_signature(test_df, prediction)

    mlflow_flavors.orbit.log_model(
        orbit_model=dlt, 
        artifact_path=model_path, 
        signature=signature
    )

run_id = run.info.run_id
print(f"MLflow run id: {run_id}")

INFO:orbit:Sampling (PyStan) with chains: 4, cores: 8, temperature: 1.000, warmups (per chain): 225 and samples(per chain): 25.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)
2023/01/06 13:57:37 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmphq_yud6b/model/model.pkl, flavor: orbit), fall back to return ['orbit==1.1.3']. Set logging level to DEBUG to see the full traceback.


MLflow run id: 772cb6959e8c42a0844e90eceee3082c


### 3.2 Deploy pyfunc model to local REST API endpoint
- Open a terminal window and cd into `examples` directory
- In the terminal run: `mlflow models serve -m runs:/<run_id>/model --env-manager local --host <host>` 
  - where you substitute `<run_id>` by the above run id and ` <host>` by the network address to listen on (e.g. 127.0.0.1)
- More details here: https://www.mlflow.org/docs/latest/cli.html#mlflow-models-serve

### 3.3 Request predictions from local REST API endpoint
- For mor details see: https://www.mlflow.org/docs/latest/models.html#built-in-deployment-tools

 #### 3.3.1 JSON input using `dataframe_split` field with pandas DataFrame in the `split` orientation

In [42]:
host = "127.0.0.1"
url = f"http://{host}:5000/invocations"

# Transform datetime column according to: https://mlflow.org/docs/latest/models.html#encoding-complex-data
test_df_serializable = test_df.copy()
test_df_serializable["week"] = test_df_serializable["week"].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
json_data = {"dataframe_split": test_df_serializable.to_dict(orient="split")}
print(json_data)

# # Un-comment the below lines to run the prediction request
# import requests
# response = requests.post(url, json=json_data)
# response.json()

{'predictions': [{'week': '2017-07-02T00:00:00',
   'prediction_5': 12.32996933073647,
   'prediction': 12.415831378619774,
   'prediction_95': 12.505552893905719,
   'trend_5': 12.249114289984634,
   'trend': 12.337001774122264,
   'trend_95': 12.435566905984784,
   'seasonality_5': 0.023341526857371835,
   'seasonality': 0.054510641952702846,
   'seasonality_95': 0.07612376663154719,
   'regression_5': 0.004599459909616243,
   'regression': 0.026254486599328643,
   'regression_95': 0.0397756624111687},
  {'week': '2017-07-09T00:00:00',
   'prediction_5': 12.426513169782348,
   'prediction': 12.534630671657983,
   'prediction_95': 12.623806508419108,
   'trend_5': 12.234585677032554,
   'trend': 12.34529267295582,
   'trend_95': 12.444150957640982,
   'seasonality_5': 0.14638690846722338,
   'seasonality': 0.1791088377692887,
   'seasonality_95': 0.204080045567357,
   'regression_5': 0.007271859206126872,
   'regression': 0.014737131377118406,
   'regression_95': 0.022364045206419186}